In [2]:
import numpy as np
import pandas as pd
from numba import njit, jit

from math import log, sqrt, pi, exp, erf
from scipy.stats import norm

from matplotlib import pyplot as plt
from tqdm import tqdm
from time import perf_counter
import gc

""" API requirements """
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

# Get Data from Yahoo Finance

In [3]:
aapl = yf.Ticker("AAPL")

In [4]:
# get historical market data
hist = aapl.history(period="max")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.100326,0.100762,0.100326,0.100326,469033600,0.0,0.0
1980-12-15,0.095528,0.095528,0.095092,0.095092,175884800,0.0,0.0
1980-12-16,0.088548,0.088548,0.088112,0.088112,105728000,0.0,0.0
1980-12-17,0.090293,0.090729,0.090293,0.090293,86441600,0.0,0.0
1980-12-18,0.092911,0.093347,0.092911,0.092911,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2022-02-28,163.059998,165.419998,162.429993,165.119995,94869100,0.0,0.0
2022-03-01,164.699997,166.600006,161.970001,163.199997,83474400,0.0,0.0
2022-03-02,164.389999,167.360001,162.949997,166.559998,79724800,0.0,0.0


In [5]:
""" Available Dates """
options_list = aapl.options
options_list

('2022-03-11',
 '2022-03-18',
 '2022-03-25',
 '2022-04-01',
 '2022-04-08',
 '2022-04-14',
 '2022-04-22',
 '2022-05-20',
 '2022-06-17',
 '2022-07-15',
 '2022-08-19',
 '2022-09-16',
 '2022-10-21',
 '2022-11-18',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

In [6]:
opt = aapl.option_chain(options_list[0])
opt

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0   AAPL220311C00105000 2022-03-02 20:41:30+00:00   105.0      62.10  0.0   
1   AAPL220311C00110000 2022-03-04 20:28:21+00:00   110.0      53.25  0.0   
2   AAPL220311C00115000 2022-03-04 19:46:47+00:00   115.0      48.10  0.0   
3   AAPL220311C00120000 2022-03-02 20:57:19+00:00   120.0      46.80  0.0   
4   AAPL220311C00125000 2022-02-28 16:26:26+00:00   125.0      40.13  0.0   
5   AAPL220311C00130000 2022-03-04 19:22:35+00:00   130.0      33.38  0.0   
6   AAPL220311C00134000 2022-03-04 16:17:05+00:00   134.0      28.71  0.0   
7   AAPL220311C00135000 2022-03-04 16:17:05+00:00   135.0      27.73  0.0   
8   AAPL220311C00136000 2022-03-04 16:28:01+00:00   136.0      26.65  0.0   
9   AAPL220311C00137000 2022-03-04 15:27:25+00:00   137.0      26.50  0.0   
10  AAPL220311C00138000 2022-03-03 19:10:20+00:00   138.0      29.35  0.0   
11  AAPL220311C00139000 2022-03-01 15:05:42+00:00   139.0     

In [7]:
opt.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220311C00105000,2022-03-02 20:41:30+00:00,105.0,62.10,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
1,AAPL220311C00110000,2022-03-04 20:28:21+00:00,110.0,53.25,0.0,0.00,0.0,0.0,70.0,0,0.000010,True,REGULAR,USD
2,AAPL220311C00115000,2022-03-04 19:46:47+00:00,115.0,48.10,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
3,AAPL220311C00120000,2022-03-02 20:57:19+00:00,120.0,46.80,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
4,AAPL220311C00125000,2022-02-28 16:26:26+00:00,125.0,40.13,0.0,0.00,0.0,0.0,2.0,0,0.000010,True,REGULAR,USD
5,AAPL220311C00130000,2022-03-04 19:22:35+00:00,130.0,33.38,0.0,0.00,0.0,0.0,72.0,0,0.000010,True,REGULAR,USD
6,AAPL220311C00134000,2022-03-04 16:17:05+00:00,134.0,28.71,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
7,AAPL220311C00135000,2022-03-04 16:17:05+00:00,135.0,27.73,0.0,0.00,0.0,0.0,21.0,0,0.000010,True,REGULAR,USD
8,AAPL220311C00136000,2022-03-04 16:28:01+00:00,136.0,26.65,0.0,0.00,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
9,AAPL220311C00137000,2022-03-04 15:27:25+00:00,137.0,26.50,0.0,0.00,0.0,0.0,2.0,0,0.000010,True,REGULAR,USD


In [8]:
""" Closing Price - S0 """
S0 = hist.iloc[-1].Close ; print(f'S0 = {S0}')

""" Options ~6 Months from now """
opt = aapl.option_chain("2022-09-16")
opt_puts = opt.puts
opt_puts

S0 = 163.1699981689453


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL220916P00040000,2022-03-04 15:18:44+00:00,40.00,0.04,0.00,0.00,0.000000,0.000000,100.0,0.0,0.500005,False,REGULAR,USD
1,AAPL220916P00041250,2022-02-09 20:05:58+00:00,41.25,0.05,0.00,0.00,0.000000,0.000000,100.0,0.0,0.500005,False,REGULAR,USD
2,AAPL220916P00042500,2022-02-22 20:00:41+00:00,42.50,0.05,0.00,0.00,0.000000,0.000000,50.0,0.0,0.500005,False,REGULAR,USD
3,AAPL220916P00043750,2022-01-28 14:59:14+00:00,43.75,0.10,0.02,0.11,0.000000,0.000000,100.0,277.0,0.699222,False,REGULAR,USD
4,AAPL220916P00045000,2022-01-28 14:47:30+00:00,45.00,0.09,0.01,0.11,0.000000,0.000000,100.0,1636.0,0.679691,False,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,AAPL220916P00700000,2020-08-27 19:44:48+00:00,700.00,22.20,5.20,6.25,-227.800000,-91.119995,23.0,7.0,0.000010,True,REGULAR,USD
103,AAPL220916P00720000,2020-08-25 19:59:35+00:00,720.00,6.00,5.60,6.65,-258.100000,-97.728134,10.0,113.0,0.000010,True,REGULAR,USD
104,AAPL220916P00740000,2020-08-26 18:55:57+00:00,740.00,26.80,6.45,7.10,-252.499980,-90.404580,NaN,1322.0,0.000010,True,REGULAR,USD
105,AAPL220916P00900000,2020-08-27 18:51:13+00:00,900.00,424.83,425.50,429.50,0.000000,0.000000,NaN,3.0,0.000010,True,REGULAR,USD


In [11]:
""" From that list, puts ~ at the money """
opt_puts.iloc[(opt_puts['strike']-S0).abs().argsort()[:1]]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
51,AAPL220916P00165000,2022-03-04 18:30:02+00:00,165.0,16.1,0.0,0.0,0.0,0.0,6.0,0.0,0.00001,True,REGULAR,USD


In [10]:
""" Treasury Bill """
t_bill = yf.Tickers(tickers="TMUBMUSD06M") #("AAPL") # TMUBMUSD06M
t_bill.history()

R = 0.00674





[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TMUBMUSD06M: No data found, symbol may be delisted


KeyError: 'TMUBMUSD06M'